In [ ]:
import os
from glob import glob

import dicom2nifti
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import numpy
from tqdm import tqdm

import torch
import torchio as tio
import torchvision
import skimage
from skimage import color

from torch.utils.data import DataLoader
import nibabel as nib

In [ ]:
dicom_input = "/home/uc/rsna-miccai-brain-tumor-radiogenomic-classification/train"
folder = glob(dicom_input + '/*')

pat_list = list()

for patient_id in tqdm(folder):
    mri_types = glob(patient_id+'/*')
    for patient in mri_types:
        name = os.path.basename(os.path.dirname(patient))
        #print("NAME: "+name)
        #print("PATIENT: "+patient)
        dir_name = os.path.basename(patient)
        dicom_input = dicom2nifti.common.read_dicom_directory(patient)
        pat_list.append(name+"_"+dir_name+"_"+str(dicom_input[0].PixelSpacing))
        #print(name, dicom_input[0].PixelSpacing)
        #print(dicom_input[0].SpacingBetweenSlices)
pat_list.sort()
print(pat_list)

In [ ]:
file_error = list()

# Convert DICOM to NIFTI by reporting errors

dicom_input = "/home/uc/rsna-miccai-brain-tumor-radiogenomic-classification/train"
nifti_output = "/home/uc/RSNA-BTC-Datasets/train_nii"
folder = glob(dicom_input + '/*')

for patient_id in tqdm(folder):
    mri_types = glob(patient_id+'/*')
    for patient in mri_types:
        name = os.path.basename(os.path.dirname(patient))
        #print("NAME: "+name)
        #print("PATIENT: "+patient)
        dir_name = os.path.basename(patient)
        try:
            #dicom_input = dicom2nifti.common.read_dicom_directory(patient)
            #print(dicom_input[0])
            dicom2nifti.dicom_series_to_nifti(patient, os.path.join(nifti_output, dir_name, name + '.nii.gz'))
        except dicom2nifti.exceptions.ConversionValidationError as err:
            print("Error with "+name+" in "+dir_name)
            print(err)
            file_error.append(name+" - "+dir_name+" - "+str(err))
            pass 
        #except Exception as err:
            #print("Error with "+name+" in "+dir_name)
            #print(err)
            #file_error.append(name+" - "+dir_name)
            #pass

(file_error)

In [ ]:
num_list = list()
for elem in file_error:
    num_list.append(elem.split(" - ")[0])
    
nifti_output = "/home/uc/RSNA-BTC-Datasets/train_nii"
folder = glob(nifti_output + '/*')
for mri_type in folder:
    patients = glob(mri_type + '/*')
    for patient in patients:
        if os.path.basename(patient)[:5] in num_list:
            print("Moving "+os.path.basename(patient))
            os.rename(patient, "/home/uc/RSNA-BTC-Datasets/removed_train_nii/"+os.path.basename(mri_type)+"/"+os.path.basename(patient))

In [ ]:
import csv

out_dict = {}

with open('train_labels.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        out_dict[row["BraTS21ID"]] = row["MGMT_value"]

    #print(out_dict)
    
    with open('train_labels_0.csv', mode='w') as csv_file:
        fieldnames = ['BraTS21ID', 'MGMT_value']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        writer.writeheader()
        for item in out_dict.items():
            if item[1] == "0":
                writer.writerow({"BraTS21ID": item[0], "MGMT_value": item[1]})

    with open('train_labels_1.csv', mode='w') as csv_file:
        fieldnames = ['BraTS21ID', 'MGMT_value']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        writer.writeheader()
        for item in out_dict.items():
            if item[1] == "1":
                writer.writerow({"BraTS21ID": item[0], "MGMT_value": item[1]})

In [ ]:
nifti_output = "/home/uc/RSNA-BTC-Datasets/train_nii"
folder = glob(nifti_output + '/*')

for mri_type in folder:
    if not os.path.exists(mri_type+"/0"):
        os.makedirs(mri_type+"/0")
    if not os.path.exists(mri_type+"/1"):
        os.makedirs(mri_type+"/1")
    patients = glob(mri_type + '/*')
    for patient in patients:
        if os.path.basename(patient) != "0" and os.path.basename(patient) != "1":
            value = out_dict[os.path.basename(patient)[:5]]
            print("Moving "+os.path.basename(patient)+" into "+value)
            os.rename(patient, mri_type+"/"+value+"/"+os.path.basename(patient))

In [ ]:
pat_list_2 = list()
folder = glob('/home/uc/RSNA-BTC-Datasets/train_nii/*')
for patient_id in folder:
    mri_types = glob(patient_id+'/*')
    for out in mri_types:
        o = glob(out+'/*')
        for patient in tqdm(o):
            nii = nib.load(patient)
            dims = nii.get_fdata().shape
            #print(dims)
            ss = nii.header.get_zooms()
            pat_list_2.append(os.path.basename(patient)[:5]+"_"+os.path.basename(patient_id)+"_"+str(ss)+"_"+str(dims))
            #print(os.path.basename(patient_id), os.path.basename(patient),sx, sy, sz)
            
pat_list_2.sort()
for elem in pat_list_2:
    print(elem)

In [ ]:
test_file_error = list()

# Convert DICOM to NIFTI by reporting errors

dicom_input = "/home/uc/rsna-miccai-brain-tumor-radiogenomic-classification/test"
nifti_output = "/home/uc/RSNA-BTC-Datasets/test_nii"
folder = glob(dicom_input + '/*')

for patient_id in tqdm(folder):
    mri_types = glob(patient_id+'/*')
    for patient in mri_types:
        name = os.path.basename(os.path.dirname(patient))
        #print("NAME: "+name)
        #print("PATIENT: "+patient)
        dir_name = os.path.basename(patient)
        try:
            #dicom_input = dicom2nifti.common.read_dicom_directory(patient)
            #print(dicom_input[0])
            dicom2nifti.dicom_series_to_nifti(patient, os.path.join(nifti_output, dir_name, name + '.nii.gz'))
        except dicom2nifti.exceptions.ConversionValidationError as err:
            print("Error with "+name+" in "+dir_name)
            print(err)
            test_file_error.append(name+" - "+dir_name+" - "+str(err))
            pass 
        #except Exception as err:
            #print("Error with "+name+" in "+dir_name)
            #print(err)
            #file_error.append(name+" - "+dir_name)
            #pass

(test_file_error)

In [ ]:
nifti_output = "/home/uc/RSNA-BTC-Datasets/test_nii"
folder = glob(nifti_output + '/*')

for mri_type in folder:
    if not os.path.exists(mri_type+"/unknown"):
        os.makedirs(mri_type+"/unknown")
    patients = glob(mri_type + '/*')
    for patient in patients:
        if os.path.basename(patient) != "unknown":
            print("Moving "+os.path.basename(patient)+" into unknown")
            os.rename(patient, mri_type+"/unknown/"+os.path.basename(patient))